In [1]:
pip install pandas scikit-learn transformers


  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.2.0
    Uninstalling fsspec-2022.2.0:
      Successfully uninstalled fsspec-2022.2.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install python-docx


Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
from docx import Document

def extract_questions_answers(doc_path):
    doc = Document(doc_path)
    data = []
    for para in doc.paragraphs:
        if para.text.startswith("Q"):
            question, answer = para.text.split(":", 1)
            data.append([question.strip(), answer.strip()])
    return pd.DataFrame(data, columns=["Question", "Correct Answer"])

# Load the data
qa_df = extract_questions_answers("C:/Users/parth/Downloads/Write Definition of the following.docx")


In [8]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation and numbers
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Apply preprocessing
qa_df['Correct Answer'] = qa_df['Correct Answer'].apply(preprocess_text)


In [14]:
pip install sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [18]:
!pip show numpy


Name: numpy
Version: 1.22.4
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: c:\users\parth\anaconda3\lib\site-packages
Requires: 
Required-by: tensorflow-intel, tensorboard, opt-einsum, ml-dtypes, keras, h5py, xarray, visions, transformers, tifffile, tables, statsmodels, stanio, sentence-transformers, seaborn, scipy, scikit-learn, scikit-image, PyWavelets, pyerfa, prophet, patsy, pandas, opencv-python, numexpr, numba, mkl-random, mkl-fft, matplotlib, linear-tree, imageio, imagecodecs, hvplot, holoviews, gensim, datashape, datashader, daal4py, cmdstanpy, Bottleneck, bokeh, bkcharts, astropy


In [21]:
pip install --upgrade numpy


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tf-keras


Note: you may need to restart the kernel to use updated packages.


In [34]:
pip show tensorflow


Name: tensorflow
Version: 2.17.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\parth\anaconda3\lib\site-packages
Requires: tensorflow-intel
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install --upgrade tensorflow


Note: you may need to restart the kernel to use updated packages.


In [5]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load the pre-trained model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Example data frame
data = {'Correct Answer': ['This is the correct answer']}
qa_df = pd.DataFrame(data)

def evaluate_answer_bert(user_answer, correct_answer):
    # Encode the sentences
    embeddings = model.encode([user_answer, correct_answer])
    
    # Compute the cosine similarity
    cosine_sim = util.pytorch_cos_sim(embeddings[0], embeddings[1])
    
    return cosine_sim.item()

# Define user_answer
user_answer = "This is a sample answer."

# Example usage
similarity_score = evaluate_answer_bert(user_answer, qa_df['Correct Answer'][0])
print(f"Similarity Score: {similarity_score}")

# Define a threshold
threshold = 0.7

# Compare similarity score with threshold
if similarity_score > threshold:
    print("The answer is similar to the correct answer.")
else:
    print("The answer is not similar to the correct answer.")


Similarity Score: 0.5573562979698181
The answer is not similar to the correct answer.


In [11]:
pip install torch


Note: you may need to restart the kernel to use updated packages.


In [6]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    return torch.mean(outputs.last_hidden_state, 1).detach()

def evaluate_answer_bert(user_answer, correct_answer):
    # Get embeddings
    user_emb = get_embeddings(user_answer)
    correct_emb = get_embeddings(correct_answer)
    
    # Compute Cosine Similarity
    cosine_sim = torch.nn.functional.cosine_similarity(user_emb, correct_emb)
    return cosine_sim.item()

# Example usage:
similarity_score = evaluate_answer_bert(user_answer, qa_df['Correct Answer'][0])

if similarity_score >= threshold:
    print("Correct Answer")
else:
    print("Incorrect Answer")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\parth\anaconda3\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\parth\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/bert-base-uncased/68d45e234eb4a928074dfd868cead0219ab85354cc53d20e772753c6bb9169d3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1724594037&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNDU5NDAzN319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9iZXJ0LWJhc2UtdW5jYXNlZC82OGQ0NWUyMzRlYjRhOTI4MDc0ZGZkODY4Y2VhZDAyMTlhYjg1MzU0Y2M1M2QyMGU3NzI3NTNjNmJiOTE2OWQzP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=j36%7E06bNYqaWLUvh2INSPvcr-3B0cjDMDIdCV-ad4Ehzu7GcY5OvykX-WK7tM0Uki%7EjQJZ15JF%7EFNGBiBoJU83ik9OTGGBXWjm%7E0c9PD%7EHzhQdsfI3beA2Q%7EedFjq592x4LNG4rhDeiJ6pYMqDD%7EV0MjRK7samZ5XsartM6n3%7EESWdnUq14iI0RKp9tzNGsawcmzW5A2aez19EbrO03ExFP05X2EQqYvhKSFjw95I5iAfjRt59WOi%7EsARdg3rqSCbfobXeUXByXCVawVuf3ZExtPNHld4CucXrztGXriezS72byte-v2lZTP4eIm-7XqZ0axso%7ET5405PPPedPt0DA__&Key-Pair-Id=K3ESJI6DHP

model.safetensors:  50%|####9     | 220M/440M [00:00<?, ?B/s]

Correct Answer


In [9]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load the pre-trained model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Example data frame with correct answers
data = {'Correct Answer': [
    'An accident is an event that can cause injury or damage.',
    'Authority to proceed is the permission for a train to enter a section.'
]}
qa_df = pd.DataFrame(data)

def evaluate_answer(user_answer, correct_answer):
    # Encode the sentences
    embeddings = model.encode([user_answer, correct_answer])
    
    # Compute the cosine similarity
    cosine_sim = util.pytorch_cos_sim(embeddings[0], embeddings[1])
    
    return cosine_sim.item()

# List of test answers
test_answers = [
    "Accident is an event that can cause injury or damage.",
    "Authority to proceed is the permission for a train to enter a section.",
]

# Evaluate each test answer
for idx, test_answer in enumerate(test_answers):
    if idx < len(qa_df):
        correct_answer = qa_df['Correct Answer'][idx]
        score = evaluate_answer(test_answer, correct_answer)
        print(f"Q{idx+1}: {test_answer} -> Score: {score}")
    else:
        print(f"Q{idx+1}: No correct answer available for comparison.")


Q1: Accident is an event that can cause injury or damage. -> Score: 0.9889760613441467
Q2: Authority to proceed is the permission for a train to enter a section. -> Score: 0.9999998807907104
